In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from pprint import pprint

sys.path.append(str(Path("../").resolve()))
pprint(sys.path)

import df_file_interchange as fi

['/home/peterma/exio_dev/repo/df_file_interchange/scratch',
 '/home/peterma/miniconda3/envs/df-file-interchange/lib/python311.zip',
 '/home/peterma/miniconda3/envs/df-file-interchange/lib/python3.11',
 '/home/peterma/miniconda3/envs/df-file-interchange/lib/python3.11/lib-dynload',
 '',
 '/home/peterma/miniconda3/envs/df-file-interchange/lib/python3.11/site-packages',
 '/home/peterma/exio_dev/repo/df_file_interchange']


In [2]:
df = pd.DataFrame(np.random.randn(3, 4), columns=["a", "b", "c", "d"])
df["pop"] = pd.array([1234, 5678, 101010])
df

,a,b,c,d,pop
0,0.997224,-0.961770,-0.959555,0.203895,1234
1,-0.390809,-0.472942,1.104313,-0.268812,5678
2,1.046016,-0.217292,-0.607914,-0.687597,101010


In [3]:
unit_cur_a = fi.ci.unit.currency.FICurrencyUnit(unit_desc="USD", unit_multiplier=1000)
unit_cur_b = fi.ci.unit.currency.FICurrencyUnit(unit_desc="EUR", unit_multiplier=1000)
unit_cur_c = fi.ci.unit.currency.FICurrencyUnit(
    unit_desc="JPY", unit_multiplier=1000000
)
unit_cur_d = fi.ci.unit.currency.FICurrencyUnit(unit_desc="USD", unit_multiplier=1000)
unit_pop = fi.ci.unit.population.FIPopulationUnit(
    unit_desc="people", unit_multiplier=1.0
)

extra_info = fi.ci.structured.FIStdExtraInfo(author="Spud", source="A simple test")

custom_info = fi.ci.structured.FIStructuredCustomInfo(
    extra_info=extra_info,
    col_units={
        "a": unit_cur_a,
        "b": unit_cur_b,
        "c": unit_cur_c,
        "d": unit_cur_d,
        "pop": unit_pop,
    },
)

In [4]:
metafile = fi.write_df_to_csv(df, Path("./test_with_structured_custom_info.csv"), custom_info=custom_info)

In [ ]:
context = {
    "clss_custom_info": {
        "FIStructuredCustomInfo": fi.ci.structured.FIStructuredCustomInfo,
    }
    "clss_extra_info": {
        "FIStdExtraInfo": fi.ci.structured.FIStdExtraInfo,
    },
    "clss_col_units": {
    
    },
}
(df_reload, metainfo_reload) = fi.read_df(metafile,)